# **Load Libraries**

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#saving and loading
import scipy.io as spio

#visualization
from matplotlib import pyplot
import math
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Input, Dense

from keras.layers import Masking
from keras.layers import Dropout
from keras import backend as K
from keras.layers import Bidirectional
from keras.callbacks import EarlyStopping, TerminateOnNaN
from keras.models import Model
from keras import optimizers
import h5py
from keras.layers import concatenate
from keras import regularizers
import keras
import random
import tensorflow as tf
from keras.engine import InputSpec

import os
#from google.colab import drive

Using TensorFlow backend.
/home/karpatne/anaconda3/envs/tfgpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/karpatne/anaconda3/envs/tfgpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/karpatne/anaconda3/envs/tfgpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/karpatne/an

# **Define Dropout Prediction Class**

In [2]:
class KerasDropoutPrediction(object):
    def __init__(self ,model):
        self.f = K.function(
            [model.layers[0].input,
             K.learning_phase()],
            [model.layers[-1].output])
    def predict(self ,x, n_iter=10):
        result = []
        for _ in range(n_iter):
            result.append(self.f([x , 1]))
        result = np.array(result).reshape(n_iter ,x.shape[0] ,x.shape[1]).T
        return result


# Mount Drive

In [3]:
#drive.mount('/content/gdrive')
#os.chdir("gdrive/My Drive/Colab Notebooks/Lake Temperature Modelling/")

# Define Custom Loss functions

In [4]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))


def masked_mean_squared_error(y_true ,y_pred):
    return K.mean(y_true[:, :, 1] * K.square(y_pred[:, :, 0] - y_true[:, :, 0]), axis=-1)


def masked_root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(y_true[:, :, 1] * K.square(y_pred[:, :, 0] - y_true[:, :, 0]), axis=-1))
  
def combined_loss(params):
    udendiff, lam = params

    def loss(y_true, y_pred):
        return masked_mean_squared_error(y_true, y_pred) + lam * K.mean(K.relu(udendiff))

    return loss


def phy_loss_mean(params):
    # useful for cross-checking training
    udendiff, lam = params

    def loss(y_true, y_pred):
        return K.mean(K.relu(udendiff))

    return loss

# Density Conversion Functions

In [5]:
def density(temp):
    return np.log10(1 + (1 - (temp + 288.9414) * (temp - 3.9863) ** 2 / (508929.2 * (temp + 68.12963))))
    # [math.log(0.1+y) for y in x]
    # return y;

def actual_density(temp):
    return (1 - (temp + 288.9414) * (temp - 3.9863) ** 2 / (508929.2 * (temp + 68.12963)))

# Load the Monotonic LSTM script

In [6]:
import sys
sys.path.append("/home/karpatne/Documents/Lake Temperature Modelling/Monotonic LSTM Script")
#import final_mLSTM
from final_mLSTM import mLSTM

# Read Data

In [7]:
def read_Data():
  
  dir='/home/karpatne/Documents/Lake Temperature Modelling/Datasets/';
  if use_temporal_feature==1:
      filename = 'temporal_mendota_train_test_split_4_year_train_new.mat';
  else:
      filename = 'new_data_mendota_train_test_split_4_year_train.mat';
  matfile = spio.loadmat(dir + filename, squeeze_me=True,
                         variable_names=['train_X','train_Y_glm','train_Y_true','test_X','test_Y_glm','test_Y_true'])

  train_X = matfile['train_X'];
  train_y = matfile['train_Y_true'];
  train_y_glm = matfile['train_Y_glm'];

  test_X = matfile['test_X'];
  test_y = matfile['test_Y_true'];
  test_y_glm = matfile['test_Y_glm'];
  params = [train_X,train_y,train_y_glm,test_X,test_y,test_y_glm];
  return params;


# Train Subset Creation

In [8]:
def train_subset(train_X, train_y, train_y_glm):
  
  count=np.zeros(train_y.shape[0]);
  for i in range(train_y.shape[0]):
      count[i]=np.count_nonzero(train_y[i,:,1]);
  tot_size=np.sum(count);  
  tr_size=tr_frac*tot_size/100;
  index=(np.arange(train_y.shape[0])).tolist();
  ix=[];
  size=0;
  while len(index)>0:
      temp=index.pop(random.randrange(0,len(index)));
      size=size+count[temp];
      ix.append(temp);
      if size>=tr_size:
          break;
  ix=np.sort(np.asarray(ix));
  train_X=train_X[ix];
  train_y=train_y[ix];
  train_y_glm=train_y_glm[ix];
  params=[ix,train_X,train_y,train_y_glm];
  return params;

## 3D to 2D transform matrix

In [9]:
def transform_3d_to_2d(X):
  return X.reshape((X.shape[0]*X.shape[1],X.shape[2]));

## 2D to 3D transform matrix using depth steps

In [10]:
def transform_2d_to_3d(X, steps):
  return X.reshape((int(X.shape[0]/steps),steps,X.shape[1]));

## Masking Output Labels

In [11]:
def masking_output_labels(
    y_density,
    y_temperature,
    mask_value):
  
  
  for i in range(y_density.shape[0]):
        if math.isnan(y_density[i, 0]):
            y_density[i, 0] = mask_value;
            y_temperature[i, 0] = mask_value;
  params=[y_density,y_temperature];
  return params;

# Edge Padding


In [12]:
def edge_padding(
    X,
    y_density,
    y_temperature,
    pad_steps):
  
  
  X_pad = np.zeros((X.shape[0], X.shape[1] + pad_steps, X.shape[2]))
  y_density_pad = np.zeros((y_density.shape[0], y_density.shape[1] + pad_steps, y_density.shape[2]))
  y_temperature_pad = np.zeros((y_temperature.shape[0], y_temperature.shape[1] + pad_steps, y_temperature.shape[2]))
  for i in range(X.shape[0]):
      X_pad[i, :, :] = np.pad(X[i, :, :], ((pad_steps, 0), (0, 0)), 'edge')
      y_density_pad[i, :, :] = np.pad(y_density[i, :, :], ((pad_steps, 0), (0, 0)), 'edge')
      y_temperature_pad[i, :, :] = np.pad(y_temperature[i, :, :], ((pad_steps, 0), (0, 0)), 'edge')
  params=[X_pad,y_density_pad,y_temperature_pad];
  return params;

# Create Model

In [13]:
def createModel(
        input_shape1,
        input_shape2,
        lstm_nodes, 
        lstm_bias, 
        drop_frac, 
        feedforward_nodes, 
        lamda_reg, 
        n_nodes):
  
  
  main_input=Input(shape=(input_shape1, input_shape2), name='main_input')
  mlstm_out=mLSTM(lstm_nodes,return_sequences=True, use_bias=lstm_bias, recurrent_dropout=drop_frac)(main_input)
  dense1=Dense(feedforward_nodes, activation='tanh',use_bias=1,kernel_regularizer=regularizers.l1_l2(l1=lamda_reg,l2=lamda_reg))(mlstm_out)
  #dense2=Dense(feedforward_nodes, activation='linear',use_bias=1,kernel_regularizer=regularizers.l1_l2(l1=0.05,l2=0.05))(dense1)
  rho_out=Dense(1, activation='linear',name='aux_output')(dense1)
  #auxiliary_input=Input(shape=(aux_train_X.shape[1],aux_train_X.shape[2]),name='aux_input')
  x=concatenate([rho_out,main_input])
  dense3=Dense(n_nodes, activation='tanh',use_bias=1,kernel_regularizer=regularizers.l1_l2(l1=lamda_reg,l2=lamda_reg))(x)
  #dense4=Dense(n_nodes, activation='linear',use_bias=1,kernel_regularizer=regularizers.l1_l2(l1=0.05,l2=0.05))(dense3)
  main_output=Dense(1, activation='linear',name='main_output')(dense3)
  
  model=Model(inputs=[main_input],outputs=[main_output,rho_out])
  return model;

# Without Dropout Prediction

In [14]:
def normal_prediction( model, test_X, test_y , pad_steps):
  
  test_pred=model.predict({'main_input':test_X})
  test_pred=test_pred[0];
  test_pred=test_pred[:,pad_steps:,:];
  test_y=test_y[:,pad_steps:,:];
  test_rmse=np.mean(np.sqrt(np.divide(np.sum(np.multiply(test_y[:,:,1],
                                                          np.square(test_pred[:,:,0]-test_y[:,:,0])),axis=1),
                                       np.sum(test_y[:,:,1],axis=1))))
  params = [test_rmse, test_pred, test_y];
  return params;

# Dropout Prediction

In [15]:
def dropout_prediction( model, n_iter, test_X, test_y , pad_steps):
  
  kdp = KerasDropoutPrediction(model);
  test_pred = kdp.predict(test_X, n_iter=n_iter)
  test_pred = test_pred[pad_steps:,:,:]
  
  test_pred_uq_mean = test_pred.mean(axis=-1).transpose()
  test_pred_uq_std = test_pred.std(axis=-1).transpose()
  test_rmse_dropout=np.mean(np.sqrt(np.divide(np.sum(np.multiply(test_y[:,:,1],
                                                                 np.square(test_pred_uq_mean[:,:]-test_y[:,:,0])),axis=1),
                                              np.sum(test_y[:,:,1],axis=1))));
  params = [test_rmse_dropout, test_pred, test_y];
  return params;
    

# Inconsitency without dropout

In [16]:
def normal_physical_inconsistency(tol, test_pred, depth_steps):
  
  test_density=actual_density(test_pred);
  test_count=np.zeros(test_density.shape[0]);
  for i in range(test_density.shape[0]):
      for j in range(test_density.shape[1]-1):
          if test_density[i,j]-test_density[i,j+1]>tol:
              test_count[i]=test_count[i]+1;
      test_count[i]=test_count[i]/depth_steps;

  test_incon=np.sum(test_count)/test_density.shape[0];
  return test_incon;

# Inconsistency of all MC samples

In [17]:
def physical_inconsistency_all_sample(tol, test_pred_do, depth_steps):
  
  test_pred_do=np.swapaxes(test_pred_do,0,2);
  density_test_pred_do=actual_density(test_pred_do);
  
  test_count_uq=np.zeros(test_pred_do.shape[0]);
  for k in range(density_test_pred_do.shape[0]):
      test_count=np.zeros(density_test_pred_do.shape[1])
      for i in range(density_test_pred_do.shape[1]):
          for j in range(density_test_pred_do.shape[2]-1):
              if density_test_pred_do[k,i,j]-density_test_pred_do[k,i,j+1]>tol:
                  test_count[i]=test_count[i]+1;
          test_count[i]=test_count[i]/depth_steps;
      test_count_uq[k]=np.sum(test_count)/density_test_pred_do.shape[1];
  test_incon_uq=np.sum(test_count_uq)/density_test_pred_do.shape[0];


  test_incon_uq=np.sum(test_count_uq)/density_test_pred_do.shape[0];
  return test_incon_uq;

# Physics Guided Architecture LSTM class

In [18]:
def PGA_LSTM_train_test(
      iteration,
      usePad,
      tr_frac,
      val_frac,
      patience_val,
      num_epochs,
      batch_size,
      lstm_nodes,
      feedforward_nodes,
      mask_value, 
      drop_frac,
      lstm_bias,
      n_nodes,
      use_GLM,
      use_temporal_feature,
      lamda_reg ):
  
  
    #Start of function
    pad_steps = 10;
    
    #read Data
    [train_X,train_y_temperature,train_y_glm,test_X,test_y_temperature,test_y_glm]=read_Data();
    
    #Add GLM feature
    if use_GLM==1:
        train_X=np.dstack((train_X,train_y_glm));
        test_X=np.dstack((test_X,test_y_glm));
    
    depth_steps=train_X.shape[1];
    
    #Normalise Data
    train_X=transform_3d_to_2d(train_X);
    test_X=transform_3d_to_2d(test_X);
    
    m1_train_mean = train_X.mean(axis=0);
    m1_train_std = train_X.std(axis=0);
    train_X = (train_X - m1_train_mean) / m1_train_std;
    test_X = (test_X - m1_train_mean) / m1_train_std;
    
    train_X=transform_2d_to_3d(train_X,depth_steps);
    test_X=transform_2d_to_3d(test_X,depth_steps);
    
    #Create train subset
    [ix,train_X,train_y_temperature,train_y_glm]=train_subset(train_X,train_y_temperature, train_y_glm);
    
    #creating path and filename for storage of results
    exp_name = 'pga_lstm_model_' + str(num_epochs)  + '_dropout_frac_' + str(drop_frac) + '_feedforward_nodes' + str(feedforward_nodes) + '_lstm_nodes' + str(
        lstm_nodes) + '_val_frac' + str(val_frac) + '_trfrac' + str(tr_frac) + '_iter' + str(iteration)+'_n_nodes'+str(n_nodes)+'_use_GLM'+str(use_GLM)+'_use_temporal_features'+str(use_temporal_feature) + '_lamda_reg'+str(lamda_reg)
    exp_name = exp_name.replace('.', 'pt')
    results_dir = '/home/karpatne/Documents/Lake Temperature Modelling/Results/paa_lstm_glm/'
    model_name = results_dir + exp_name + '_model.h5'  # storing the trained model
    results_name = results_dir + exp_name + '_results.mat'  # storing the results of the model
    
    train_y_temperature=transform_3d_to_2d(train_y_temperature);
    test_y_temperature=transform_3d_to_2d(test_y_temperature);
    
    #Create Density labels 
    train_y_density=np.zeros(np.shape(train_y_temperature));
    test_y_density=np.zeros(np.shape(test_y_temperature));
    train_y_density[:,1], test_y_density[:,1] = train_y_temperature[:,1], test_y_temperature[:,1]
    train_y_density[:,0], test_y_density[:,0] = density(train_y_temperature[:,0]), density(test_y_temperature[:,0])
    
    # density output normalisation
    mean = np.nanmean(train_y_density[:, 0]);
    std = np.nanstd(train_y_density[:, 0]);
    train_y_density[:, 0] = (train_y_density[:, 0] - mean) / std;
    test_y_density[:, 0] = (test_y_density[:, 0] - mean) / std;
    
    #masking output labels
    [train__y_density,train_y_temperature]=masking_output_labels(train_y_density,train_y_temperature,mask_value);
    [test__y_density,test_y_temperature]=masking_output_labels(test_y_density,test_y_temperature,mask_value);
    
    #reshaping train/test output density/temperature labels
    train_y_density=transform_2d_to_3d(train_y_density,depth_steps); 
    train_y_temperature=transform_2d_to_3d(train_y_temperature,depth_steps); 
    test_y_density=transform_2d_to_3d(test_y_density,depth_steps); 
    test_y_temperature=transform_2d_to_3d(test_y_temperature,depth_steps); 
    
    #perform edge padding
    [train_X_pad,train_y_density_pad,train_y_temperature_pad]=edge_padding(train_X,train_y_density,train_y_temperature,pad_steps);
    [test_X_pad,test_y_density_pad,test_y_temperature_pad]=edge_padding(test_X,test_y_density,test_y_temperature,pad_steps);
    
    #create auxiliary dataset (same as input data)
    aux_train_X=train_X_pad;
    aux_test_X=test_X_pad;
    
    model=createModel(
        train_X_pad.shape[1],
        train_X_pad.shape[2],
        lstm_nodes, 
        lstm_bias, 
        drop_frac, 
        feedforward_nodes, 
        lamda_reg, 
        n_nodes);
    
    model.summary();
    
    optimiser = optimizers.Adadelta(clipnorm=3)
    #optimiser = optimizers.Adam(clipnorm=3)
    
    model.compile(
        loss={'main_output': masked_mean_squared_error, 'aux_output': masked_mean_squared_error}, 
        optimizer=optimiser,
        loss_weights={'main_output':1.,'aux_output':0.2})
    
    
    early_stopping = EarlyStopping(
                            monitor='val_main_output_loss', 
                            patience=patience_val, 
                            verbose=1)
    history=model.fit(
                {'main_input':train_X_pad},
                {'main_output':train_y_temperature_pad,'aux_output':train_y_density_pad},
                epochs=num_epochs, 
                batch_size=batch_size, 
                verbose=2, 
                shuffle=False,
                validation_split=val_frac, 
                callbacks=[early_stopping, TerminateOnNaN()]);
    
    #Calculating model RMSE 
    [test_rmse1, test_pred1, test_y1] = normal_prediction(model, test_X_pad, test_y_temperature_pad, pad_steps);
    [train_rmse1, train_pred1, train_y1] = normal_prediction(model, train_X_pad, train_y_temperature_pad, pad_steps);
    
    print('Without dropout = TrainRMSE : ' +str(train_rmse1) + ' TestRMSE : '+str(test_rmse1));
    
    [test_rmse_dropout, test_pred_do, test_y1] = dropout_prediction(model,100,test_X_pad, test_y_temperature_pad[:,pad_steps:,:], pad_steps);
    [train_rmse_dropout, train_pred_do, train_y1] = dropout_prediction(model,100,train_X_pad, train_y_temperature_pad[:,pad_steps:,:], pad_steps);
    
    print('With dropout = TrainRMSE : ' +str(train_rmse_dropout) + ' TestRMSE : '+str(test_rmse_dropout));
    
    
    #Calculating Physical Inconsistencies
    tol=0.0001;
    test_inconsistency_without_dropout=normal_physical_inconsistency(tol, test_pred1, depth_steps);
    train_inconsistency_without_dropout=normal_physical_inconsistency(tol, train_pred1, depth_steps);
    
    print("Without dropout : Test Incon = "+str(test_inconsistency_without_dropout)+'  Train Incon = '+str(train_inconsistency_without_dropout))
    
    test_pred_uq_mean = test_pred_do.mean(axis=-1).transpose();
    train_pred_uq_mean = train_pred_do.mean(axis=-1).transpose();
    test_inconsistency_dropout_mean=normal_physical_inconsistency(tol, test_pred_uq_mean, depth_steps);
    train_inconsistency_dropout_mean=normal_physical_inconsistency(tol, train_pred_uq_mean, depth_steps);
    
    print("With dropout Inconsistency of sample mean: Test Incon = "+str(test_inconsistency_dropout_mean)+'  Train Incon = '+str(train_inconsistency_dropout_mean))
    
    test_inconsistency_dropout_all=physical_inconsistency_all_sample(tol, test_pred_do, depth_steps);
    train_inconsistency_dropout_all=physical_inconsistency_all_sample(tol, train_pred_do, depth_steps);
    
    print("With dropout Inconsistency of sample mean: Test Incon = "+str(test_inconsistency_dropout_all)+'  Train Incon = '+str(train_inconsistency_dropout_all))
    model.save(model_name)
    spio.savemat(results_name,{'test_pred_temperature':test_pred1,
                               'test_y_temperature':test_y1,
                               'train_pred_temperature':train_pred1, 
                               'train_y_temperature':train_y1,
                               'train_rmse':train_rmse1,
                               'test_rmse':test_rmse1,
                               'train_rmse_dropout':train_rmse_dropout,
                               'test_rmse_dropout':test_rmse_dropout,
                               'test_incon_without_dropout':test_inconsistency_without_dropout,
                               'train_inconsistency_without_dropout':train_inconsistency_without_dropout,
                               'test_inconsistency_dropout_mean':test_inconsistency_dropout_mean,
                               'train_inconsistency_dropout_mean':train_inconsistency_dropout_mean,
                               'test_inconsistency_dropout_all':test_inconsistency_dropout_all,
                               'train_inconsistency_dropout_all':train_inconsistency_dropout_all,
                               'test_predictions_all':test_pred_do,
                               'train_predictions_all':train_pred_do,
                               'train_index':ix,
                               'train_main_loss':history.history['main_output_loss'],
                               'train_aux_output_loss':history.history['aux_output_loss'],
                               'train_loss':history.history['loss'], 
                               'train_val_loss':history.history['val_loss']})
    

# Main function

In [19]:
if __name__ == '__main__':
	# Main Function
    #tr_frac_range = [50,25,20,15,10,5];
    tr_frac_range=[100,50,25,10,5];
    #tr_frac=50;
    val_frac = 0.1;
    patience_val = 100;
    num_epochs = 3000;
    batch_size = 20;
    #lstm_nodes_range = [4,8,16,25];
    lstm_nodes=8;
    #feedforward_nodes_range = [3,5,7,10];
    feedforward_nodes=5;
    mask_value = 0;
    drop_frac = 0.2;
    #drop_frac_range=[0.2,0.3,0.5];
    lstm_bias = 1;
    #n_nodes_range=[5,10];
    n_nodes=5;
    use_GLM=1;
    use_temporal_feature=1;
    usePad = 0;
    iter_range=np.arange(5);
    lamda_reg=0.01;
    #iter_range=[1];
    #iteration=0;  
    
    for iteration in iter_range:
       for tr_frac in tr_frac_range:
            PGA_LSTM_train_test(
                          iteration,
                          usePad,
                          tr_frac,
                          val_frac,
                          patience_val,
                          num_epochs,
                          batch_size,
                          lstm_nodes,
                          feedforward_nodes,
                          mask_value,
                          drop_frac,
                          lstm_bias,
                          n_nodes,
                          use_GLM,
                          use_temporal_feature,
                          lamda_reg
            )





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 60, 17)       0                                            
__________________________________________________________________________________________________
m_lstm_1 (mLSTM)                (None, 60, 8)        832         main_input[0][0]                 
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 60, 5)        45          m_lstm_1[0][0]                   
__________________________________________________________________________________________________
aux_output (Dense)              (None, 60, 1)        6           dense_1

Epoch 26/3000
 - 1s - loss: 24.2009 - main_output_loss: 23.1259 - aux_output_loss: 3.6071 - val_loss: 10.4470 - val_main_output_loss: 8.9098 - val_aux_output_loss: 5.9251
Epoch 27/3000
 - 1s - loss: 22.6778 - main_output_loss: 21.7106 - aux_output_loss: 3.0813 - val_loss: 10.7878 - val_main_output_loss: 9.2924 - val_aux_output_loss: 5.7273
Epoch 28/3000
 - 1s - loss: 21.5854 - main_output_loss: 20.7081 - aux_output_loss: 2.6433 - val_loss: 10.9654 - val_main_output_loss: 9.5137 - val_aux_output_loss: 5.5217
Epoch 29/3000
 - 1s - loss: 21.0710 - main_output_loss: 20.2074 - aux_output_loss: 2.5847 - val_loss: 9.7418 - val_main_output_loss: 8.5711 - val_aux_output_loss: 4.1252
Epoch 30/3000
 - 1s - loss: 20.3436 - main_output_loss: 19.4868 - aux_output_loss: 2.5621 - val_loss: 11.6699 - val_main_output_loss: 10.2554 - val_aux_output_loss: 5.3597
Epoch 31/3000
 - 1s - loss: 19.6856 - main_output_loss: 18.8589 - aux_output_loss: 2.4252 - val_loss: 11.1667 - val_main_output_loss: 9.8236 - va

Epoch 75/3000
 - 1s - loss: 4.3609 - main_output_loss: 3.9771 - aux_output_loss: 0.5192 - val_loss: 2.1224 - val_main_output_loss: 1.6884 - val_aux_output_loss: 0.7719
Epoch 76/3000
 - 1s - loss: 4.2184 - main_output_loss: 3.8382 - aux_output_loss: 0.5090 - val_loss: 2.0074 - val_main_output_loss: 1.5707 - val_aux_output_loss: 0.7949
Epoch 77/3000
 - 1s - loss: 4.1927 - main_output_loss: 3.8187 - aux_output_loss: 0.4879 - val_loss: 1.9261 - val_main_output_loss: 1.4941 - val_aux_output_loss: 0.7789
Epoch 78/3000
 - 1s - loss: 3.9866 - main_output_loss: 3.6159 - aux_output_loss: 0.4802 - val_loss: 1.8513 - val_main_output_loss: 1.4314 - val_aux_output_loss: 0.7273
Epoch 79/3000
 - 1s - loss: 3.9678 - main_output_loss: 3.6001 - aux_output_loss: 0.4726 - val_loss: 1.9893 - val_main_output_loss: 1.5825 - val_aux_output_loss: 0.6690
Epoch 80/3000
 - 1s - loss: 3.7762 - main_output_loss: 3.4114 - aux_output_loss: 0.4645 - val_loss: 2.0296 - val_main_output_loss: 1.6156 - val_aux_output_loss:

Epoch 124/3000
 - 1s - loss: 1.4976 - main_output_loss: 1.2468 - aux_output_loss: 0.2316 - val_loss: 1.1886 - val_main_output_loss: 0.8972 - val_aux_output_loss: 0.4336
Epoch 125/3000
 - 1s - loss: 1.5230 - main_output_loss: 1.2733 - aux_output_loss: 0.2298 - val_loss: 1.2080 - val_main_output_loss: 0.9175 - val_aux_output_loss: 0.4332
Epoch 126/3000
 - 1s - loss: 1.4817 - main_output_loss: 1.2330 - aux_output_loss: 0.2292 - val_loss: 1.1922 - val_main_output_loss: 0.9025 - val_aux_output_loss: 0.4344
Epoch 127/3000
 - 1s - loss: 1.4167 - main_output_loss: 1.1700 - aux_output_loss: 0.2247 - val_loss: 1.1818 - val_main_output_loss: 0.8987 - val_aux_output_loss: 0.4073
Epoch 128/3000
 - 1s - loss: 1.4404 - main_output_loss: 1.1944 - aux_output_loss: 0.2261 - val_loss: 1.1342 - val_main_output_loss: 0.8532 - val_aux_output_loss: 0.4010
Epoch 129/3000
 - 1s - loss: 1.4214 - main_output_loss: 1.1765 - aux_output_loss: 0.2239 - val_loss: 1.1612 - val_main_output_loss: 0.8826 - val_aux_output

Epoch 173/3000
 - 1s - loss: 0.9546 - main_output_loss: 0.7418 - aux_output_loss: 0.2187 - val_loss: 1.2547 - val_main_output_loss: 1.0147 - val_aux_output_loss: 0.3558
Epoch 174/3000
 - 1s - loss: 0.9433 - main_output_loss: 0.7315 - aux_output_loss: 0.2165 - val_loss: 1.2426 - val_main_output_loss: 1.0045 - val_aux_output_loss: 0.3488
Epoch 175/3000
 - 1s - loss: 0.9494 - main_output_loss: 0.7384 - aux_output_loss: 0.2161 - val_loss: 1.2487 - val_main_output_loss: 1.0102 - val_aux_output_loss: 0.3559
Epoch 176/3000
 - 1s - loss: 0.9487 - main_output_loss: 0.7387 - aux_output_loss: 0.2153 - val_loss: 1.0495 - val_main_output_loss: 0.8125 - val_aux_output_loss: 0.3512
Epoch 177/3000
 - 1s - loss: 0.9605 - main_output_loss: 0.7509 - aux_output_loss: 0.2167 - val_loss: 1.0920 - val_main_output_loss: 0.8553 - val_aux_output_loss: 0.3531
Epoch 178/3000
 - 1s - loss: 0.9379 - main_output_loss: 0.7286 - aux_output_loss: 0.2181 - val_loss: 1.0678 - val_main_output_loss: 0.8311 - val_aux_output

...

Epoch 27/3000
 - 0s - loss: 141.8641 - main_output_loss: 141.2132 - aux_output_loss: 1.0374 - val_loss: 42.2505 - val_main_output_loss: 41.5355 - val_aux_output_loss: 1.3581
Epoch 28/3000
 - 0s - loss: 140.7381 - main_output_loss: 140.1040 - aux_output_loss: 0.9539 - val_loss: 41.8479 - val_main_output_loss: 41.1221 - val_aux_output_loss: 1.4132
Epoch 29/3000
 - 0s - loss: 139.4541 - main_output_loss: 138.8065 - aux_output_loss: 1.0230 - val_loss: 41.3689 - val_main_output_loss: 40.6285 - val_aux_output_loss: 1.4877
Epoch 30/3000
 - 0s - loss: 139.3111 - main_output_loss: 138.6520 - aux_output_loss: 1.0813 - val_loss: 40.9196 - val_main_output_loss: 40.1701 - val_aux_output_loss: 1.5346
Epoch 31/3000
 - 0s - loss: 137.8711 - main_output_loss: 137.2013 - aux_output_loss: 1.1364 - val_loss: 40.4885 - val_main_output_loss: 39.7349 - val_aux_output_loss: 1.5563
Epoch 32/3000
 - 0s - loss: 136.3046 - main_output_loss: 135.6201 - aux_output_loss: 1.2115 - val_loss: 40.0091 - val_main_output_

Epoch 75/3000
 - 0s - loss: 91.7820 - main_output_loss: 90.6873 - aux_output_loss: 3.2197 - val_loss: 16.7899 - val_main_output_loss: 15.6750 - val_aux_output_loss: 3.3206
Epoch 76/3000
 - 0s - loss: 88.8410 - main_output_loss: 87.7508 - aux_output_loss: 3.1973 - val_loss: 15.9095 - val_main_output_loss: 14.7808 - val_aux_output_loss: 3.3897
Epoch 77/3000
 - 0s - loss: 86.4108 - main_output_loss: 85.2591 - aux_output_loss: 3.5052 - val_loss: 15.0981 - val_main_output_loss: 13.9549 - val_aux_output_loss: 3.4630
Epoch 78/3000
 - 0s - loss: 87.0177 - main_output_loss: 85.9241 - aux_output_loss: 3.2147 - val_loss: 14.5214 - val_main_output_loss: 13.3733 - val_aux_output_loss: 3.4877
Epoch 79/3000
 - 0s - loss: 84.7763 - main_output_loss: 83.6455 - aux_output_loss: 3.4013 - val_loss: 13.7201 - val_main_output_loss: 12.5606 - val_aux_output_loss: 3.5455
Epoch 80/3000
 - 0s - loss: 83.2704 - main_output_loss: 82.1168 - aux_output_loss: 3.5158 - val_loss: 13.3849 - val_main_output_loss: 12.218

Epoch 123/3000
 - 0s - loss: 43.3438 - main_output_loss: 41.8887 - aux_output_loss: 5.0547 - val_loss: 3.0441 - val_main_output_loss: 1.6345 - val_aux_output_loss: 4.8294
Epoch 124/3000
 - 0s - loss: 42.5985 - main_output_loss: 41.1778 - aux_output_loss: 4.8847 - val_loss: 3.0579 - val_main_output_loss: 1.6469 - val_aux_output_loss: 4.8379
Epoch 125/3000
 - 0s - loss: 42.0026 - main_output_loss: 40.6026 - aux_output_loss: 4.7835 - val_loss: 3.0685 - val_main_output_loss: 1.6569 - val_aux_output_loss: 4.8415
Epoch 126/3000
 - 0s - loss: 41.4784 - main_output_loss: 40.0184 - aux_output_loss: 5.0843 - val_loss: 3.0988 - val_main_output_loss: 1.6859 - val_aux_output_loss: 4.8505
Epoch 127/3000
 - 0s - loss: 40.6393 - main_output_loss: 39.2143 - aux_output_loss: 4.9114 - val_loss: 3.1426 - val_main_output_loss: 1.7280 - val_aux_output_loss: 4.8614
Epoch 128/3000
 - 0s - loss: 39.9059 - main_output_loss: 38.4950 - aux_output_loss: 4.8431 - val_loss: 3.1993 - val_main_output_loss: 1.7828 - va

Epoch 171/3000
 - 0s - loss: 18.9464 - main_output_loss: 17.6923 - aux_output_loss: 4.2085 - val_loss: 12.0120 - val_main_output_loss: 10.7767 - val_aux_output_loss: 4.1176
Epoch 172/3000
 - 0s - loss: 19.5650 - main_output_loss: 18.3155 - aux_output_loss: 4.1895 - val_loss: 12.1513 - val_main_output_loss: 10.9198 - val_aux_output_loss: 4.1006
Epoch 173/3000
 - 0s - loss: 18.4458 - main_output_loss: 17.1772 - aux_output_loss: 4.2875 - val_loss: 12.3841 - val_main_output_loss: 11.1617 - val_aux_output_loss: 4.0606
Epoch 174/3000
 - 0s - loss: 18.2515 - main_output_loss: 16.9288 - aux_output_loss: 4.5630 - val_loss: 12.6264 - val_main_output_loss: 11.4137 - val_aux_output_loss: 4.0176
Epoch 175/3000
 - 0s - loss: 17.7648 - main_output_loss: 16.4963 - aux_output_loss: 4.2980 - val_loss: 12.8031 - val_main_output_loss: 11.6035 - val_aux_output_loss: 3.9581
Epoch 176/3000
 - 0s - loss: 17.6061 - main_output_loss: 16.3596 - aux_output_loss: 4.1937 - val_loss: 13.0240 - val_main_output_loss: 

Epoch 219/3000
 - 0s - loss: 10.4284 - main_output_loss: 9.5045 - aux_output_loss: 2.7438 - val_loss: 7.8423 - val_main_output_loss: 6.9430 - val_aux_output_loss: 2.6225
Epoch 220/3000
 - 0s - loss: 10.3626 - main_output_loss: 9.4231 - aux_output_loss: 2.8251 - val_loss: 7.5528 - val_main_output_loss: 6.6597 - val_aux_output_loss: 2.5949
Epoch 221/3000
 - 0s - loss: 10.1366 - main_output_loss: 9.2023 - aux_output_loss: 2.8027 - val_loss: 7.3389 - val_main_output_loss: 6.4513 - val_aux_output_loss: 2.5708
Epoch 222/3000
 - 0s - loss: 10.6683 - main_output_loss: 9.7296 - aux_output_loss: 2.8264 - val_loss: 7.1621 - val_main_output_loss: 6.2775 - val_aux_output_loss: 2.5559
Epoch 00222: early stopping
Without dropout = TrainRMSE : 3.097607758054841 TestRMSE : 4.594692101989104
With dropout = TrainRMSE : 3.0770720711644524 TestRMSE : 4.396901588830412
Without dropout : Test Incon = 0.0011621621621621624  Train Incon = 0.0
With dropout Inconsistency of sample mean: Test Incon = 0.0001621621